<a href="https://colab.research.google.com/github/cedro3/others/blob/master/StyleCLIP_global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
%tensorflow_version 1.x
! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
! pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
! git clone https://github.com/orpatashnik/StyleCLIP

In [ ]:
# input dataset name 
dataset_name='ffhq' # input dataset name, currently, only support ffhq

% cd StyleCLIP/global/

# input prepare data 
!python GetCode.py --dataset_name $dataset_name --code_type 'w'
!python GetCode.py --dataset_name $dataset_name --code_type 's'
!python GetCode.py --dataset_name $dataset_name --code_type 's_mean_std'

import tensorflow as tf
import numpy as np 
import torch
import clip
from PIL import Image
import pickle
import copy
import matplotlib.pyplot as plt
from MapTS import GetFs,GetBoundary,GetDt
from manipulate import Manipulator
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

M=Manipulator(dataset_name='ffhq')
fs3=np.load('./npy/ffhq/fs3.npy')
np.set_printoptions(suppress=True)

# GUIによる画像編集

In [ ]:
# 画像選択
img_indexs=[1259]
dlatent_tmp=[tmp[img_indexs] for tmp in M.dlatents]
M.num_images=len(img_indexs)

In [ ]:
#テキスト入力
neutral='face'
target='smile face'
classnames=[target,neutral]
dt=GetDt(classnames,model)

In [ ]:
#@markdown ###元画像表示
beta = 0.1 
alpha = 0 
M.alpha=[alpha]
boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
codes=M.MSCode(dlatent_tmp,boundary_tmp2)
out=M.GenerateImg(codes)
Image.fromarray(out[0,0])

In [ ]:
#@markdown ###編集画像表示
beta = 0.1 #@param {type:"slider", min:0.08, max:0.3, step:0.01}
alpha = 2 #@param {type:"slider", min:-10, max:10, step:0.1}
M.alpha=[alpha]
boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
codes=M.MSCode(dlatent_tmp,boundary_tmp2)
out=M.GenerateImg(codes)
Image.fromarray(out[0,0])

# 画像編集ビデオ

In [ ]:
# 画像の選択
img_indexs=[1276]
dlatent_tmp=[tmp[img_indexs] for tmp in M.dlatents]
M.num_images=len(img_indexs)

In [ ]:
# テキスト入力
neutral='face with hair'
target='Curly Hair'
classnames=[target,neutral]
dt=GetDt(classnames,model)

In [ ]:
# 段階的な編集画像の保存
import os
import shutil
if os.path.isdir('pic'):
     shutil.rmtree('pic')
os.makedirs('pic', exist_ok=True)
cnt = 0
for i in range(0,20,1):
     beta = 0.1 
     alpha = i/10 
     M.alpha=[alpha]
     boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
     codes=M.MSCode(dlatent_tmp,boundary_tmp2)
     out=M.GenerateImg(codes)
     pic = Image.fromarray(out[0,0])
     pic.save('./pic/'+str(cnt).zfill(6)+'.png')     
     cnt +=1

for i in range(20,0,-1):
     beta = 0.1 
     alpha = i/10 
     M.alpha=[alpha]
     boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
     codes=M.MSCode(dlatent_tmp,boundary_tmp2)
     out=M.GenerateImg(codes)
     pic = Image.fromarray(out[0,0])
     pic.save('./pic/'+str(cnt).zfill(6)+'.png')     
     cnt +=1     

In [ ]:
# 段階的な編集画像を動画に変換
! ffmpeg -r 10 -i pic/%6d.png\
              -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# Play the generated video
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('output.mp4')